In [1]:
#@title Install packages
# !pip install -q torch
!pip install scanpy
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install leidenalg python-igraph
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=4fb871178771aed124c9db75fc3ea67e03ea9a9d857ddf9749de0485ab94418a
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.2 MB/s eta 0:00:00


In [2]:
#@title Importing packages
import scanpy as sc
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

import os
# import torch
# os.environ['TORCH'] = torch.__version__
# os.environ['PYTHONWARNINGS'] = "ignore"

# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv, global_mean_pool

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from sklearn import preprocessing
import scanpy as sc

import anndata

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch_geometric.nn import GCNConv


from collections import Counter

from sklearn.cluster import KMeans
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import AgglomerativeClustering
import os


import leidenalg
import igraph as ig
from scipy.spatial import distance
import numpy as np

import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx


import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [3]:
#@title Connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Reading Files
class Read_File:
    def __init__(self, file_path):
        self.file_path = file_path

    def getFilePath(self):
        return self.file_path

    def readFileRetDF(self):
        df = pd.read_csv(self.file_path)
        return df

    def readFileByScanpy(self):
        adata = sc.read(self.file_path)
        adata = adata.T
        return adata

In [5]:
#@title AutoEncoder
class AutoEncoder:
    def __init__(self, input_dim, encoding_dim):
        self.input_dim = input_dim
        self.encoding_dim = encoding_dim

    def add_noise(self, x_train):
        noise_factor = 0.5
        x_train = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
        return x_train

    def performEncoding(self, x_train_noise, x_train):
        input_img = Input(shape=(self.input_dim,))
        # stacked autoencoder
        # encoded = Dense(4096, activation='relu')(input_img)
        # encoded = Dense(2048, activation='relu')(encoded)
        # encoded = Dense(1024, activation='relu')(input_img)
        # encoded = Dense(512, activation='relu')(encoded)
        # encoded = Dense(264, activation='relu')(input_img)
        # encoded = Dense(128, activation='relu')(encoded)
        # encoded = Dense(64, activation='relu')(encoded)
        # "encoded" is the encoded representation of the input
        encoded = Dense(self.encoding_dim, activation='relu')(input_img)
        # stacked autoencoder
        # decoded = Dense(64, activation='relu')(encoded)
        # decoded = Dense(128, activation='relu')(decoded)
        # decoded = Dense(264, activation='relu')(decoded)
        # decoded = Dense(512, activation='relu')(decoded)
        # decoded = Dense(1024, activation='relu')(decoded)
        # decoded = Dense(2048, activation='relu')(decoded)
        # decoded = Dense(4096, activation='relu')(decoded)
        # "decoded" is the lossy reconstruction of the input
        decoded = Dense(self.input_dim, activation='sigmoid')(decoded)

        # This model maps an input to its reconstruction
        autoencoder = Model(input_img, decoded)
        # This model maps an input to its encoded representation
        encoder = Model(input_img, encoded)
        # decoder = Model(encoded, decoded)

        autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
        # Now let's train our autoencoder for 50 epochs:

        autoencoder.fit(x_train_noise, x_train,
                        epochs=50,
                        batch_size=x_train.shape[0],
                        shuffle=True, verbose = 0)
        # encoding the data
        encoded_imgs = encoder.predict(x_train, verbose=0)
        # decoded_imgs = decoder.predict(encoded_imgs)
        return encoded_imgs

    def performDecoding(self, x_train_noise, x_train):
        input_img = Input(shape=(self.input_dim,))
        # stacked autoencoder
        # encoded = Dense(4096, activation='relu')(input_img)
        # encoded = Dense(2048, activation='relu')(encoded)
        # encoded = Dense(1024, activation='relu')(input_img)
        # encoded = Dense(512, activation='relu')(encoded)
        # encoded = Dense(264, activation='relu')(input_img)
        # encoded = Dense(128, activation='relu')(encoded)
        # encoded = Dense(64, activation='relu')(encoded)
        # encoded = Dense(100, activation='relu')(encoded)
        # "encoded" is the encoded representation of the input
        encoded = Dense(self.encoding_dim, activation='relu')(input_img)

        # stacked autoencoder
        # decoded = Dense(100, activation='relu')(encoded)
        # decoded = Dense(64, activation='relu')(encoded)
        # decoded = Dense(128, activation='relu')(decoded)
        # decoded = Dense(264, activation='relu')(decoded)
        # decoded = Dense(512, activation='relu')(decoded)
        # decoded = Dense(1024, activation='relu')(decoded)
        # decoded = Dense(2048, activation='relu')(decoded)
        # decoded = Dense(4096, activation='relu')(decoded)
        # "decoded" is the lossy reconstruction of the input
        decoded = Dense(self.input_dim, activation='sigmoid')(encoded)

        # This model maps an input to its reconstruction
        autoencoder = Model(input_img, decoded)
        # This model maps an input to its encoded representation
        encoder = Model(input_img, encoded)

        # create a placeholder for an encoded (32-dimensional) input
        encoded_input = Input(shape=(self.encoding_dim,))
        # retrieve the last layer of the autoencoder model
        decoder_layer = autoencoder.layers[-1]
        # create the decoder model
        print("----------------------")
        decoder = Model(encoded, decoded)
        print("----------------------")

        autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
        # Now let's train our autoencoder for 50 epochs:

        autoencoder.fit(x_train_noise, x_train,
                        epochs=50,
                        batch_size=x_train.shape[0],
                        shuffle=True, verbose = 0)
        # encoding the data
        encoded_imgs = encoder.predict(x_train, verbose=0)
        decoded_imgs = decoder.predict(encoded_imgs, verbose = 0)
        return decoded_imgs


In [14]:
#@title PCA calculation OR Autoencoder

class FineTune:

    # def __init__(self, df):
    #     self.df = df

    def makeCellByGeneFile(self, df):
        df = df.drop(df.columns[0], axis=1) # removing gene names
        df = df.T # making cell * genes
        num_of_genes = df.shape[1]
        return df, num_of_genes

    def normalize(self, df):
        df = preprocessing.normalize(df)
        df = np.log1p(df)
        df = pd.DataFrame(df)
        return df


    def findHVG(self, df, num_of_genes):

        adata = anndata.AnnData(df)

        n_top_genes = 10000
        if num_of_genes >= 10000:
            n_top_genes = 10000
        else:
            n_top_genes = num_of_genes


        sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, min_mean=0.0125, max_mean=3)
        highly_variable_genes = adata.var['highly_variable']


        # print(highly_variable_genes)
        df = df.loc[:, highly_variable_genes.values]
        print("highly variable genes", df.shape)
        return df


    def PCA_for_divided_data(self, df, start, end, n_components):
        pca = PCA(n_components=.95)
        # pca = PCA(n_components=n_components)
        pca.fit(df.iloc[:, start:end])
        pca_features = pca.transform(df.iloc[:, start:end])
        # print("shape of divided data ", pca_features.shape)
        return pca_features

    def PCA4(self, df, num_of_genes, n_components, number_of_division, overlapping_percentage):

        pca_features_all = []
        current_genes = 10000
        if num_of_genes >= 10000:
            current_genes = 10000
        else:
            current_genes = num_of_genes

        ranze_size = (int)(current_genes / number_of_division)
        overlap_size = (int)(ranze_size * overlapping_percentage / 100)

        loop = 1
        for i in range(0, current_genes, ranze_size):

            if i + ranze_size > current_genes:
                break

            start = np.maximum(i - overlap_size, 0)
            end = np.minimum(i + ranze_size + overlap_size, current_genes)

            if loop == number_of_division:
                end = current_genes

            pca_features_all.append(self.PCA_for_divided_data(df, start, end, n_components))
            # print(i, i + ranze_size)
            loop = loop + 1

        return pca_features_all

    def PCA4GeneCluster(self, df, range_list, n_components, overlapping_percentage):

        pca_features_all = []

        for i in range(1, len(range_list), 1):

            start = range_list[i - 1]
            end = range_list[i]

            start = start - 200
            end = end + 200

            start = np.maximum(start, 0)
            end = np.minimum(end, df.shape[1])

            # if i == len(range_list) - 1:
            #     end = df.shape[0]

            pca_features_all.append(self.PCA_for_divided_data(df, start, end, n_components))

        return pca_features_all


    def performTuning(self, df):
        df, num_of_genes = self.makeCellByGeneFile(df)
        df = self.normalize(df)
        # print("values greater than 0 ", (df >= 0).sum().sum(), "values less than 0 ", (df < 0).sum().sum())
        df = self.findHVG(df, num_of_genes)
        return df, num_of_genes



    def performPCA(self, df, num_of_genes, n_components, number_of_division, overlapping_percentage):

        pca_features_all = self.PCA4(df, num_of_genes, n_components, number_of_division, overlapping_percentage)

        # print(pca_features_all[0].shape)
        # print(type(pca_features_all[0]))

        # pca_features_final = pca_features_all[number_of_division - 1]
        # for i in range(number_of_division - 1):
        #     pca_features_final = pca_features_final + pca_features_all[i]

        # pca_features_final = pca_features_final / number_of_division

        # just to concetenate

        pca_features_final = pca_features_all[number_of_division - 1]
        for i in range(number_of_division - 1):
            pca_features_final = np.concatenate((pca_features_final, pca_features_all[i]), axis=1)

        print("final shape of pca ", pca_features_final.shape)

        return pca_features_final

    def performPCAonClusterGene(self, df, range_list, n_components, overlapping_percentage):
        pca_features_all = self.PCA4GeneCluster(df, range_list, n_components, overlapping_percentage)

        pca_features_final = pca_features_all[len(range_list) - 2]
        for i in range(len(range_list) - 2):
            pca_features_final = np.concatenate((pca_features_final, pca_features_all[i]), axis=1)

        print("final shape of pca ", pca_features_final.shape)

        return pca_features_final

    # Autoencoders


    def Encode(self, df, num_of_genes, n_components, number_of_division, overlapping_percentage):

        pca_features_all = []
        current_genes = 10000
        if num_of_genes >= 10000:
            current_genes = 10000
        else:
            current_genes = num_of_genes

        ranze_size = (int)(current_genes / number_of_division)
        overlap_size = (int)(ranze_size * overlapping_percentage / 100)

        loop = 1
        for i in range(0, current_genes, ranze_size):

            if i + ranze_size > current_genes:
                break

            start = np.maximum(i - overlap_size, 0)
            end = np.minimum(i + ranze_size + overlap_size, current_genes)

            if loop == number_of_division:
                end = current_genes

            autoencoder = AutoEncoder(end - start, n_components)
            noised_data = autoencoder.add_noise(np.array(df.iloc[:, start:end].copy()))

            pca_features_all.append(autoencoder.performEncoding(noised_data, np.array(df.iloc[:, start:end].copy())))
            # print(i, i + ranze_size)
            loop = loop + 1

        return pca_features_all



    def performEncoding(self, df, num_of_genes, n_components, number_of_division, overlapping_percentage):

        pca_features_all = self.Encode(df, num_of_genes, n_components, number_of_division, overlapping_percentage)

        # print(pca_features_all[0].shape)

        # pca_features_final = pca_features_all[number_of_division - 1]
        # for i in range(number_of_division - 1):
        #     pca_features_final = pca_features_final + pca_features_all[i]

        # pca_features_final = pca_features_final / number_of_division

        # just to concetenate

        pca_features_final = pca_features_all[number_of_division - 1]
        for i in range(number_of_division - 1):
            pca_features_final = np.concatenate((pca_features_final, pca_features_all[i]), axis=1)

        # print("final shape of pca ", pca_features_final.shape)

        return pca_features_final

    # bucket method
    def PCA4Buckets(self, df, list_of_buckets, n_components, overlapping_percentage):

            pca_features_all = []
            for i in range(len(list_of_buckets)):
                temp_df = df.iloc[:, list_of_buckets[i]]
                pca = PCA(n_components=.95)
                pca.fit(temp_df)
                pca_features = pca.transform(temp_df)
                pca_features_all.append(pca_features)

            return pca_features_all

    def performPCAonBuckets(self, df, number_of_division, n_components, overlapping_percentage):

        big_bucket_size = df.shape[1]
        list_of_buckets = []
        for i in range(number_of_division):
            list_of_buckets.append([])
        global_list = []
        count = 0

        while( len(global_list) != big_bucket_size ):
            count = count + 1
            # print("sesh ki hobe--?----------------------------", count, len(global_list))
            picked_random_gene = np.random.randint(0, big_bucket_size)
            picked_random_bucket = np.random.randint(0, number_of_division)
            if (picked_random_gene in list_of_buckets[picked_random_bucket]):
                continue
            else:
                list_of_buckets[picked_random_bucket].append(picked_random_gene)
                if picked_random_gene not in global_list:
                    global_list.append(picked_random_gene)


        print("BUCKETS ARE FILLED----------------------")

        pca_features_all = self.PCA4Buckets(df, list_of_buckets, n_components, overlapping_percentage)

        pca_features_final = pca_features_all[number_of_division - 1]
        for i in range(number_of_division - 1):
            pca_features_final = np.concatenate((pca_features_final, pca_features_all[i]), axis=1)

        print("final shape of pca ", pca_features_final.shape)

        return pca_features_final

        return pca_features_all

In [7]:

#@title Common class "Models" having common functions
class Models:

    def __init__(self, f_Tune, ground_truth, model_name):
        self.f_Tune = f_Tune
        self.ground_truth = ground_truth
        self.model_name = model_name
    def plot(self, pca_features_final, data_name, folder_name, number_of_division, ARI, labels):
        print("ekhane ase????????????????????????????????????????????")
        # colors = ["g.", "r.", "b.", "y."]
        # checking
        colors = ["g.", "r.", "b.", "y.", "c.", "m.", "k.", "w.", "g.", "r.", "b.", "y.", "c.", "m.", "k.", "w.", "g.", "r.", "b.", "y."]
        for i in range(len(pca_features_final)):
            plt.plot(pca_features_final[i][0], pca_features_final[i][1], colors[labels[i]], markersize=10)
        # plt.scatter(centroids[:,0], centroids[:,1], marker='x', s=150, linewidths=5)
        plt.xlabel('principle component 1')
        plt.ylabel('principle component 2')
        # accuracy based on ARI

        # create a folder
        folder_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/plot_{self.model_name}'
        folder_path1 = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/plot_{self.model_name}/pca_20'

        if not os.path.exists(folder_path):
            # Create the directory if it does not exist
            os.mkdir(folder_path)

        else:
            print("Directory already exists.")

        if not os.path.exists(folder_path1):
            print('++++++++++++++++++++++++++')
            # Create the directory if it does not exist
            os.mkdir(folder_path1)

        else:
            print('+-----------------------------')
            print("Directory already exists.")

        plt.title(f'ARI = {ARI}')
        # plt.plot()
        plt.savefig(f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/plot_{self.model_name}/pca_20/division_{number_of_division}.png')
        plt.show()
        plt.clf()
        plt.close()

In [8]:
#@title  Hierarchical clustering
class Hierarchical_clustering(Models):

    # def __init__(self, f_Tune, ground_truth):
    #     self.f_Tune = f_Tune
    #     self.ground_truth = ground_truth

    def performClustering(self, pca_features_final, n_clusters):

        hierarchical = AgglomerativeClustering(n_clusters=n_clusters)
        labels = hierarchical.fit_predict(pca_features_final)

        # print(f'label counts {len(self.labels)} --- ground_truth {len(self.ground_truth)}')
        ARI = metrics.adjusted_rand_score(list(labels), ground_truth)
        print(f'ari - {ARI}')

        return ARI, labels

In [9]:
#@title  Kmeans

class KMEANS(Models):
    # def __init__(self, f_Tune, ground_truth):
    #     self.f_Tune = f_Tune
    #     self.ground_truth = ground_truth

    def performClustering(self, pca_features_final, n_clusters):

        kmeans = KMeans(n_clusters=n_clusters)
        kmeans.fit(pca_features_final)

        print(pca_features_final.shape)

        #get the centroids
        centroids = kmeans.cluster_centers_
        labels = kmeans.labels_
        # print(f'label counts {len(self.labels)} --- ground_truth {len(self.ground_truth)}')
        ARI = metrics.adjusted_rand_score(list(labels), ground_truth)
        print(f'ari - {ARI}')

        return ARI, labels

In [15]:
#@title  Data Info

list_of_datas = ['yan', 'pollen', 'kolodziejczyk', 'intestine', 'goolam', 'fan', 'deng-rpkms']
cluster_info = [2, 1, 1, 2, 1, 1, 1]

# list_of_datas = ['intestine']
# cluster_info = [2]


# list_of_datas = ['GSE138852']
# cluster_info = [8]

In [ ]:
#@title number of clusters estimation by silhouette_score

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score



def silhouette_method(data):
    silhouette_scores = []
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
        score = silhouette_score(data, kmeans.labels_)
        silhouette_scores.append(score)

    # return the index of max silhouette score
    max_index = np.argmax(silhouette_scores)
    return max_index + 2

def calinski_harabasz_method(data):
    cluster_range = range(2, 15)
    ch_scores = []

    # Calculate the CH Index for each number of clusters
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(data)
        labels = kmeans.labels_
        ch_score = calinski_harabasz_score(data, labels)
        ch_scores.append(ch_score)

    max_index = np.argmax(ch_scores)
    return max_index + 2

# cluster_count = []
# for ind in range(len(list_of_datas)):

#     print(f'datasets --------------------------------------------------------------- {ind}')

#     data_name = list_of_datas[ind]


#     file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_data.csv'



#     #####################
#     df = Read_File(file_path_data).readFileRetDF()
#     fTune = FineTune()
#     df, num_of_genes = fTune.performTuning(df)

#     cluster_count.append(calinski_harabasz_method(np.array(df)))

# print(cluster_count)



In [ ]:
#@title Generalized for all datasets for any type of clustering

randomization = True
n_components = 50
model_name = "KMEANS"
overlapping_percentage = 20
data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}_random_ov{overlapping_percentage}_variance95(2).csv'
# data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}.csv'
output_in_csv = pd.DataFrame()



# initialize output_in_csv with division
division_no = []

for i in range(1, 21, 1):
    division_no.append(f'division {i}')

output_in_csv["division no"] = division_no


for ind in range(len(list_of_datas)):

    print(f'datasets --------------------------------------------------------------- {ind}')

    data_name = list_of_datas[ind]
    col_no_for_cluster_info = cluster_info[ind]
    folder_name = "plot_HI_cluster"


    file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_data.csv'
    file_path_celldata = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_celldata.csv'

    # actual data count in each cluster
    df = Read_File(file_path_celldata).readFileRetDF()
    # column number whaere the cluster information stored(col_no_for_cluster_info)
    lis = list(df.iloc[:, col_no_for_cluster_info])
    # ground_truth = list(np.sort(list(dict(Counter(lis)).values())))

    #number of cluster
    n_cluster = len(list(np.sort(list(dict(Counter(lis)).values()))))
    print(f'number of clusters {n_cluster}')

    label_encoder = LabelEncoder()
    ground_truth = label_encoder.fit_transform(lis)
    print(len(ground_truth))

    min_error = +100000000
    final_division = -1
    data_name = data_name

    #####################
    df = Read_File(file_path_data).readFileRetDF()
    fTune = FineTune()
    df, num_of_genes = fTune.performTuning(df)
    if(randomization):
        df = df[np.random.permutation(df.columns)]



    list_ARI = []
    # exploring different division on data
    for i in range(1, 21, 1):

      print(f'division --------------------------------------------------------------- {i}')

      number_of_division = i
      pca_features_final = fTune.performPCA(df, num_of_genes, n_components, number_of_division, overlapping_percentage)

      km = KMEANS(fTune, ground_truth, model_name)
      ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      #save the results
      list_ARI.append(ARI)
      # km.plot(pca_features_final = pca_features_final, data_name = data_name, folder_name = folder_name, number_of_division = number_of_division, ARI = ARI, labels = labels)

    output_in_csv[data_name] = list_ARI

output_in_csv.to_csv(data_vs_ARI_path, index = False)

    #   if(km.error < min_error):
    #       min_error = km.error
    #       final_division = i

    # print("final division", final_division)

In [ ]:
#@title sanity checking for treutlein data

file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/goolam/goolam_data.csv'
df = Read_File(file_path_data).readFileRetDF()
fTune = FineTune()
df, num_of_genes = fTune.performTuning(df)
print(df.shape)

In [ ]:
#@title Gene clustering approach for cell clustering
randomization = False
n_components = 50
model_name = "KMEANS"
overlapping_percentage = 20
data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}_clusteringGene3.csv'
# data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}.csv'
output_in_csv = pd.DataFrame()

# range_list = [0, 2939, 5825, 8353, 10000]

# initialize output_in_csv with division
division_no = []

for i in range(1, 3, 1):
    division_no.append(f'division {i}')

output_in_csv["division no"] = division_no


for ind in range(len(list_of_datas)):

    print(f'datasets --------------------------------------------------------------- {ind}')

    data_name = list_of_datas[ind]
    col_no_for_cluster_info = cluster_info[ind]
    folder_name = "plot_HI_cluster"


    file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_data.csv'
    file_path_celldata = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_celldata.csv'
    file_path_geneCluster_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_gene_cluster_data.csv'

    # actual data count in each cluster
    df = Read_File(file_path_celldata).readFileRetDF()
    # column number whaere the cluster information stored(col_no_for_cluster_info)
    lis = list(df.iloc[:, col_no_for_cluster_info])
    # ground_truth = list(np.sort(list(dict(Counter(lis)).values())))

    #number of cluster
    n_cluster = len(list(np.sort(list(dict(Counter(lis)).values()))))
    print(f'number of clusters {n_cluster}')

    label_encoder = LabelEncoder()
    ground_truth = label_encoder.fit_transform(lis)
    print(len(ground_truth))

    min_error = +100000000
    final_division = -1
    data_name = data_name


    # read gene cluster data
    df_geneCluster = pd.read_csv(file_path_geneCluster_data)

    # read gene expression data
    df = Read_File(file_path_data).readFileRetDF()
    fTune = FineTune()
    df, num_of_genes = fTune.performTuning(df)

    print("shape after fine tune ", df.shape)

    df = df.T

    # df['cluster'] = df_geneCluster['cluster']
    # for i in range(df.shape[0]):
    #     df['cluster'].iloc[i] = int(df_geneCluster['cluster'].iloc[i])


    print("shape after transpose ", df.shape)
    # Calculate k nearest neighbors --------------------------------------
    # print("calculating neighbors using KNN")
    k = (int)(np.sqrt(df.shape[0]) / 2)
    nbrs = NearestNeighbors(n_neighbors=k).fit(df)
    distances, indices = nbrs.kneighbors(df)
    # print("making graphs using KNN", indices.shape)
    # --------------------------------------------------------

    # graph creation-----------------------------------
    G = nx.Graph()
    for ii in range(len(df)):
        for jj in range(len(indices[ii])):
            if ii != indices[ii][jj] and distances[ii][jj] != 0:  # Avoid self-loops
                weight = 1.0 / distances[ii][jj]  # Inverse of distance as edge weight
                G.add_edge(ii, indices[ii][jj], weight=weight)

    ig_graph = ig.Graph.from_networkx(G)
    # ----------------------------------------------------------------------

    # partition or clustering and saving the data ----------------------------
    partition = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition)

    file_path_geneCluster_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_gene_cluster_data.csv'
    labels_of_genes = partition.membership
    labels_of_genes_df = pd.DataFrame(labels_of_genes, columns=['cluster'])
    labels_of_genes_df.to_csv(file_path_geneCluster_data, index = False)
    df['cluster'] = labels_of_genes
    # ----------------------------------------------------------------------

    df.sort_values('cluster', inplace=True)

    range_list = [0]
    for ii in range(1, df.shape[0], 1):
        if df['cluster'].iloc[ii] != df['cluster'].iloc[ii-1]:
            range_list.append(ii)
    range_list.append(df.shape[0])

    print("range list00------------------------------", range_list)

    df.drop('cluster', axis=1, inplace=True)
    df = df.T


    list_ARI = []
    # exploring different division on data
    for i in range(1, 3, 1):

      print(f'division --------------------------------------------------------------- {i}')

      number_of_division = i
      if i == 1:
          pca_features_final = fTune.performPCA(df, num_of_genes, n_components, number_of_division, overlapping_percentage)
      else:
          pca_features_final = fTune.performPCAonClusterGene(df, range_list, n_components, overlapping_percentage)

      km = KMEANS(fTune, ground_truth, model_name)
      ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      #save the results
      list_ARI.append(ARI)
      # km.plot(pca_features_final = pca_features_final, data_name = data_name, folder_name = folder_name, number_of_division = number_of_division, ARI = ARI, labels = labels)

    output_in_csv[data_name] = list_ARI

output_in_csv.to_csv(data_vs_ARI_path, index = False)

    #   if(km.error < min_error):
    #       min_error = km.error
    #       final_division = i

    # print("final division", final_division)

In [ ]:
#@title Generalized for all datasets for any type of clustering - WITH AUTOENCODER

randomization = True
n_components = 50
model_name = "KMEANS"
overlapping_percentage = 20
data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}_random_ov{overlapping_percentage}_Encoder3.csv'
# data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}.csv'
output_in_csv = pd.DataFrame()


# initialize output_in_csv with division
division_no = []

for i in range(1, 21, 1):
    division_no.append(f'division {i}')

output_in_csv["division no"] = division_no


for ind in range(len(list_of_datas)):

    print(f'datasets --------------------------------------------------------------- {ind}')

    data_name = list_of_datas[ind]
    col_no_for_cluster_info = cluster_info[ind]
    folder_name = "plot_HI_cluster"


    file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_data.csv'
    file_path_celldata = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_celldata.csv'

    # actual data count in each cluster
    df = Read_File(file_path_celldata).readFileRetDF()
    # column number whaere the cluster information stored(col_no_for_cluster_info)
    lis = list(df.iloc[:, col_no_for_cluster_info])
    # ground_truth = list(np.sort(list(dict(Counter(lis)).values())))

    #number of cluster
    n_cluster = len(list(np.sort(list(dict(Counter(lis)).values()))))
    print(f'number of clusters {n_cluster}')

    label_encoder = LabelEncoder()
    ground_truth = label_encoder.fit_transform(lis)
    print(len(ground_truth))

    min_error = +100000000
    final_division = -1
    data_name = data_name

    #####################
    df = Read_File(file_path_data).readFileRetDF()
    fTune = FineTune()
    df, num_of_genes = fTune.performTuning(df)
    if(randomization):
        df = df[np.random.permutation(df.columns)]



    list_ARI = []
    # exploring different division on data
    for i in range(1, 21, 1):

      print(f'division --------------------------------------------------------------- {i}')

      number_of_division = i
      pca_features_final = fTune.performEncoding(df, num_of_genes, n_components, number_of_division, overlapping_percentage)

      km = KMEANS(fTune, ground_truth, model_name)
      ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      #save the results
      list_ARI.append(ARI)
      # km.plot(pca_features_final = pca_features_final, data_name = data_name, folder_name = folder_name, number_of_division = number_of_division, ARI = ARI, labels = labels)

    output_in_csv[data_name] = list_ARI

output_in_csv.to_csv(data_vs_ARI_path, index = False)

    #   if(km.error < min_error):
    #       min_error = km.error
    #       final_division = i

    # print("final division", final_division)

In [ ]:
#@title data imputing with autoencoder and then clustering

randomization = False
n_components = 32
model_name = "KMEANS"
overlapping_percentage = 20
data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/final_data/dump/bucket1.csv'
# data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{model_name}_{n_components}.csv'
output_in_csv = pd.DataFrame()
n_components_list = [50]

# range_list = [0, 2939, 5825, 8353, 10000]

# initialize output_in_csv with division
division_no = []

for i in range(1, 21, 1):
    division_no.append(f'division {i}')

output_in_csv["division no"] = division_no


for ind in range(len(list_of_datas)):

    print(f'datasets --------------------------------------------------------------- {ind}')

    data_name = list_of_datas[ind]
    col_no_for_cluster_info = cluster_info[ind]
    folder_name = "plot_HI_cluster"


    file_path_data = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_data.csv'
    file_path_celldata = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/{data_name}/{data_name}_celldata.csv'

    # actual data count in each cluster
    df = Read_File(file_path_celldata).readFileRetDF()
    # column number whaere the cluster information stored(col_no_for_cluster_info)
    lis = list(df.iloc[:, col_no_for_cluster_info])
    # ground_truth = list(np.sort(list(dict(Counter(lis)).values())))

    #number of cluster
    n_cluster = len(list(np.sort(list(dict(Counter(lis)).values()))))
    # print(f'number of clusters {n_cluster}')

    label_encoder = LabelEncoder()
    ground_truth = label_encoder.fit_transform(lis)
    print(len(ground_truth))

    min_error = +100000000
    final_division = -1
    data_name = data_name

    #####################
    df = Read_File(file_path_data).readFileRetDF()
    fTune = FineTune()
    df, num_of_genes = fTune.performTuning(df)
    if(randomization):
        df = df[np.random.permutation(df.columns)]

    # df.to_csv(f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/intestine/intestine_cell_gene.csv', index = False)

    decoded_data_whole = np.zeros((df.shape[0],df.shape[1]))

    df_np = np.array(df)
    df_imputed = df.copy()
    # imputing data
    for n_comp in n_components_list:
        autoencoder = AutoEncoder(df.shape[1], n_comp)
        noised_data = autoencoder.add_noise(np.array(df.copy()))
        decoded_data = autoencoder.performDecoding(noised_data, np.array(df.copy()))
        decoded_data_whole = decoded_data_whole + decoded_data

    decoded_data_whole = decoded_data_whole/len(n_components_list)
    # df_np = df.to_numpy()
    df_imputed_np = df_imputed.to_numpy()

    df_imputed_np = np.where(df_imputed_np == 0, decoded_data_whole, df_imputed_np)

    # number of clusters
    # n_cluster = calinski_harabasz_method(df_imputed_np)
    print("number of clusters = ", n_cluster)
    # number of zeroes count
    print('number of values in 0:1 range in original -- ', np.count_nonzero(df_np == 0))
    print('number of values in 0:1 range in imputed data -- ', np.count_nonzero((df_imputed_np == 0)))

    df_imputed = pd.DataFrame(df_imputed_np)

    # print('number of zeroes in teh original data -- ', df.isin([0]).sum().sum())
    # print('number of zeroes in the imputed data --', df_imputed.isin([0]).sum().sum())

    # for itr in range(df.shape[0]):
    #     for itr1 in range(df.shape[1]):
    #         if(df_imputed.iloc[itr, itr1] == 0):
    #             df_imputed.iloc[itr, itr1] = decoded_data[itr, itr1]

    list_ARI = []
    list_ARI_imputed = []
    # list_ARI_decoded = []
    # exploring different division on data
    for i in range(1, 21, 1):

      print(f'division --------------------------------------------------------------- {i}')

      number_of_division = i

      # pca_features_final = fTune.performPCA(df, num_of_genes, n_components, number_of_division, overlapping_percentage)
      # pca_features_final = fTune.performEncoding(df, num_of_genes, n_components, number_of_division, overlapping_percentage)
      pca_features_final = fTune.performPCAonBuckets(df, number_of_division, n_components, overlapping_percentage)
      km = KMEANS(fTune, ground_truth, model_name)
      ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      list_ARI.append(ARI)

      # for the whole imputed data

      # pca_features_final = fTune.performPCA(df_imputed, num_of_genes, n_components, number_of_division, overlapping_percentage)
      # pca_features_final = fTune.performEncoding(df_imputed, num_of_genes, n_components, number_of_division, overlapping_percentage)
      pca_features_final = fTune.performPCAonBuckets(df, number_of_division, n_components, overlapping_percentage)
      km = KMEANS(fTune, ground_truth, model_name)
      ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      list_ARI_imputed.append(ARI)

      # for the whole decocded data

      # pca_features_final = fTune.performPCA(pd.DataFrame(decoded_data), num_of_genes, n_components, number_of_division, overlapping_percentage)
      # km = KMEANS(fTune, ground_truth, model_name)
      # ARI, labels = km.performClustering(pca_features_final = pca_features_final, n_clusters = n_cluster)
      # list_ARI_decoded.append(ARI)


      # km.plot(pca_features_final = pca_features_final, data_name = data_name, folder_name = folder_name, number_of_division = number_of_division, ARI = ARI, labels = labels)

    output_in_csv[data_name] = list_ARI
    output_in_csv[f'{data_name}_imputed'] = list_ARI_imputed
    # output_in_csv[f'{data_name}_decoded'] = list_ARI_decoded

output_in_csv.to_csv(data_vs_ARI_path, index = False)

    #   if(km.error < min_error):
    #       min_error = km.error
    #       final_division = i

    # print("final division", final_division)

datasets --------------------------------------------------------------- 0
90


/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


highly variable genes (90, 10000)
----------------------
----------------------
number of clusters =  7
number of values in 0:1 range in original --  222691
number of values in 0:1 range in imputed data --  0
division --------------------------------------------------------------- 1
BUCKETS ARE FILLED----------------------
final shape of pca  (90, 16)
(90, 16)
ari - 0.5372539216036402


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


BUCKETS ARE FILLED----------------------
final shape of pca  (90, 16)
(90, 16)
ari - 0.6612969734621623
division --------------------------------------------------------------- 2


...

(704, 1545)
ari - 0.15898382385213145
BUCKETS ARE FILLED----------------------
final shape of pca  (704, 1482)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(704, 1482)
ari - 0.1636374623057967
division --------------------------------------------------------------- 8
BUCKETS ARE FILLED----------------------
final shape of pca  (704, 1698)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(704, 1698)
ari - 0.17596075779021475


In [17]:
data_vs_ARI_path = f'/content/drive/MyDrive/Research/Bioinformatics/scRNAseq/datas/final_data/dump/{model_name}_{n_components}_random_ov{overlapping_percentage}_newdata3.csv'

output_in_csv.to_csv(data_vs_ARI_path, index = False)

In [15]:
output_in_csv

,division no,yan,yan_imputed,pollen,pollen_imputed,kolodziejczyk,kolodziejczyk_imputed,intestine,intestine_imputed,goolam,goolam_imputed,fan,fan_imputed,deng-rpkms,deng-rpkms_imputed
0,division 1,0.558906,0.670759,0.827631,0.776593,0.158872,0.672292,0.420925,0.255282,0.143728,0.848938,0.079778,0.365264,0.403562,0.398713
1,division 2,0.492811,0.767353,0.810733,0.879868,0.158872,0.688206,0.362613,0.201085,0.241076,0.703999,0.134751,0.337556,0.407646,0.389870
2,division 3,0.537254,0.644097,0.812173,0.888641,0.156786,0.690259,0.383303,0.181603,0.140973,0.712872,0.089397,0.336658,0.381864,0.400401
3,division 4,0.523687,0.528268,0.818471,0.825235,0.159118,0.678183,0.424522,0.192199,0.147302,0.656601,0.079624,0.356438,0.404204,0.383505
4,division 5,0.537254,0.424550,0.807831,0.932589,0.156786,0.701260,0.422096,0.186465,0.289326,0.657508,0.089397,0.318193,0.406488,0.389229
5,division 6,0.638825,0.630826,0.732808,0.862834,0.156642,0.685546,0.437127,0.163918,0.136486,0.680813,0.099925,0.318193,0.398516,0.411894
6,division 7,0.670062,0.644561,0.826020,0.807891,0.156521,0.669747,0.389872,0.163199,0.136486,0.377862,0.100054,0.318193,0.374927,0.440480
7,division 8,0.523687,0.733546,0.844347,0.825890,0.154166,0.691624,0.365452,0.175740,0.254424,0.868658,0.096008,0.356956,0.411606,0.360781
8,division 9,0.526512,0.755747,0.816192,0.852988,0.174010,0.669747,0.344332,0.203682,0.143728,0.589970,0.170737,0.325485,0.392094,0.436763
9,division 10,0.506906,0.661814,0.861836,0.870987,0.156840,0.682650,0.472381,0.194346,0.133728,0.862729,0.077335,0.330558,0.423780,0.390068
